In [1]:
import numpy as np
import pandas as pd
import re
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import urllib.request
import json
import pickle
from eunjeon import Mecab

mecab = Mecab()
np.random.seed(seed=0)

In [2]:
data = pd.read_csv("data/full.csv")

data = data.drop(['summary'], axis=1)
data.drop_duplicates(subset=['utterance'], inplace=True)
print('전체 대화 개수 :',(len(data)))

전체 대화 개수 : 314996


In [3]:
# 전처리 함수 (필요없는 문자 제거, 형태소 분석 및 분리)
def preprocess_sentence(sentence):
    sentence = re.sub(r'\#[^)]*\#', '', sentence) # 마스킹된 데이터(이름, 날짜 등) 제거
    sentence = re.sub("[^가-힣]", " ", sentence) # ㅋㅋㅋ, 영어, 특수문자, 숫자 등 제거
    
    speech_pos = mecab.pos(sentence)
    #조사, 어미, 접두접미사 등 제거
    clear_pos = [n for n, tag in speech_pos if tag.startswith('M') | tag.startswith('N') | tag.startswith('V')]
    sentence = ' '.join(clear_pos)
    
    tokens = ' '.join(word for word in sentence.split())
    return tokens

In [4]:
# utterance(대화 본문) 전처리
clean_text = []

for s in data['utterance']:
    clean_text.append(preprocess_sentence(s))
    
data['utterance'] = clean_text

In [5]:
# 길이가 공백인 샘플은 NULL 값으로 변환 후 제거 (전처리 과정에서 모든 단어 삭제된 케이스)
data.replace('', np.nan, inplace=True)
data.dropna(axis = 0, inplace = True)
print('전체 대화 개수 :',(len(data)))

전체 대화 개수 : 314324


In [6]:
# 대화 파트 길이 측정
text_len = [len(s.split()) for s in data['utterance']]
print(np.percentile(text_len, 75) * 1.5) #maximum

# 패딩길이
text_max_len = 63 # Q3 * 1.5
summary_max_len = 4 # 카테고리 최대 단어 갯수 + 토큰

63.0


In [7]:
# 패딩 길이보다 본문이 긴 데이터는 사용하지 않음
data = data[data['utterance'].apply(lambda x: len(x.split()) <= text_max_len)]
print('전체 샘플수 :',(len(data)))

전체 샘플수 : 298121


In [8]:
# 주제 파트에는 시작 토큰과 종료 토큰을 추가
data['decoder_input'] = data['topic'].apply(lambda x : 'sostoken '+ x)
data['decoder_target'] = data['topic'].apply(lambda x : x + ' eostoken')
data.head()

,utterance,topic,decoder_input,decoder_target
0,마져 난 그래서 한 번 보 엄청 오래 보 친구 많 스탈 아냐 딱 보이 좁 깊 사귀 ...,개인 및 관계,sostoken 개인 및 관계,개인 및 관계 eostoken
1,내일 아침 먹 시 일어나 하 가능 피곤 하 그래도 먹 했 최대한 일어나 챙겨 먹 보...,개인 및 관계,sostoken 개인 및 관계,개인 및 관계 eostoken
2,이따가 저 못 할 수 잇 나 역내 저 해두 댕 왜 못 행 아빠 계시 나 저 하 아빠...,개인 및 관계,sostoken 개인 및 관계,개인 및 관계 eostoken
3,상금 천만 원 번 선택 사람 빵 글 쿠만 얼마 오려 얼마 안 나올 듯 육처 넌 너 ...,개인 및 관계,sostoken 개인 및 관계,개인 및 관계 eostoken
4,아까 둘 이 동시 울 약간 멘탈 후 달림 지금 막 수 시간 남 그거 먹이 재우 해 ...,개인 및 관계,sostoken 개인 및 관계,개인 및 관계 eostoken


In [9]:
# 학습 위해 데이터 랜덤 셔플한 뒤 학습, 테스트 데이터 스플릿
encoder_input = np.array(data['utterance'])
decoder_input = np.array(data['decoder_input'])
decoder_target = np.array(data['decoder_target'])

indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)

In [10]:
encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

n_of_val = int(len(encoder_input)*0.15)

encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

In [11]:
# 희귀 단어 통계를 위한 임시 토크나이저
src_tokenizer = Tokenizer()
src_tokenizer.fit_on_texts(encoder_input_train)

In [12]:
# 예측 시 overfitting 등 다양한 문제 야기할 수 있는 희귀 단어 제외를 위한 통계 체크
threshold = 19
total_cnt = len(src_tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in src_tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('희귀 단어를 제외시킬 경우의 단어 집합의 크기 %s'%(total_cnt - rare_cnt))
print("희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 76130
희귀 단어를 제외시킬 경우의 단어 집합의 크기 16904
희귀 단어의 비율: 77.79587547615921
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 3.0511399056988977


In [13]:
# 전체 단어 중 약 3%를 차지하는 희귀 단어 제거
# 단어 집합의 크기를 16900으로 제한하고 tokenizer를 pickle 파일로 저장
src_vocab = 16900
src_tokenizer = Tokenizer(num_words = src_vocab) 
src_tokenizer.fit_on_texts(encoder_input_train)

with open('model/tokenizer1.pickle', 'wb') as s_handle:
    pickle.dump(src_tokenizer, s_handle, protocol=pickle.HIGHEST_PROTOCOL)

# 텍스트 시퀀스를 정수 시퀀스로 변환
encoder_input_train = src_tokenizer.texts_to_sequences(encoder_input_train) 
encoder_input_test = src_tokenizer.texts_to_sequences(encoder_input_test)

In [14]:
# 단어 집합의 크기를 17으로 제한하고 (주제 파트는 사용 단어가 정해져있음) tokenizer를 pickle 파일로 저장
tar_vocab = 17
tar_tokenizer = Tokenizer(num_words = tar_vocab) 
tar_tokenizer.fit_on_texts(decoder_input_train)
tar_tokenizer.fit_on_texts(decoder_target_train)

with open('model/tokenizer2.pickle', 'wb') as t_handle:
    pickle.dump(tar_tokenizer, t_handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# 텍스트 시퀀스를 정수 시퀀스로 변환
decoder_input_train = tar_tokenizer.texts_to_sequences(decoder_input_train) 
decoder_target_train = tar_tokenizer.texts_to_sequences(decoder_target_train)
decoder_input_test = tar_tokenizer.texts_to_sequences(decoder_input_test)
decoder_target_test = tar_tokenizer.texts_to_sequences(decoder_target_test)

In [15]:
# 주제 파트에서 토큰만 남은 (사실상 NULL값) 데이터 제거
drop_train = [index for index, sentence in enumerate(decoder_input_train) if len(sentence) == 1]
drop_test = [index for index, sentence in enumerate(decoder_input_test) if len(sentence) == 1]

encoder_input_train = np.delete(encoder_input_train, drop_train, axis=0)
decoder_input_train = np.delete(decoder_input_train, drop_train, axis=0)
decoder_target_train = np.delete(decoder_target_train, drop_train, axis=0)

encoder_input_test = np.delete(encoder_input_test, drop_test, axis=0)
decoder_input_test = np.delete(decoder_input_test, drop_test, axis=0)
decoder_target_test = np.delete(decoder_target_test, drop_test, axis=0)

In [16]:
encoder_input_train = pad_sequences(encoder_input_train, maxlen = text_max_len, padding='post')
encoder_input_test = pad_sequences(encoder_input_test, maxlen = text_max_len, padding='post')
decoder_input_train = pad_sequences(decoder_input_train, maxlen = summary_max_len, padding='post')
decoder_target_train = pad_sequences(decoder_target_train, maxlen = summary_max_len, padding='post')
decoder_input_test = pad_sequences(decoder_input_test, maxlen = summary_max_len, padding='post')
decoder_target_test = pad_sequences(decoder_target_test, maxlen = summary_max_len, padding='post')

In [17]:
embedding_dim = 128
hidden_size = 256

# 인코더 입력 층
encoder_inputs = Input(shape=(text_max_len,))

# 인코더의 임베딩 층
enc_emb = Embedding(src_vocab, embedding_dim)(encoder_inputs)

# 인코더의 LSTM 1
encoder_lstm1 = LSTM(hidden_size, return_sequences=True, return_state=True ,dropout = 0.4, recurrent_dropout = 0.4)
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb)

# 인코더의 LSTM 2
encoder_lstm2 = LSTM(hidden_size, return_sequences=True, return_state=True, dropout=0.4, recurrent_dropout=0.4)
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1)

# 인코더의 LSTM 3
encoder_lstm3 = LSTM(hidden_size, return_sequences=True, return_state=True, dropout=0.4, recurrent_dropout=0.4)
encoder_outputs, state_h, state_c= encoder_lstm3(encoder_output2)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [18]:
# 디코더 입력 층
decoder_inputs = Input(shape=(None,))

# 디코더의 임베딩 층
dec_emb_layer = Embedding(tar_vocab, embedding_dim)
dec_emb = dec_emb_layer(decoder_inputs)

# 디코더의 LSTM
decoder_lstm = LSTM(hidden_size, return_sequences = True, return_state = True, dropout = 0.4, recurrent_dropout=0.4)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state = [state_h, state_c]) # 인코더의 마지막 LSTM 상태

In [19]:
# 어텐션 층 임포트
import ssl
context = ssl._create_unverified_context()

urllib.request.urlretrieve("https://raw.githubusercontent.com/thushv89/attention_keras/master/src/layers/attention.py", filename="attention.py")
from attention import AttentionLayer

In [20]:
# 어텐션 층(어텐션 함수)
attn_layer = AttentionLayer(name='attention_layer')
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs])

# 어텐션 층의 결과와 디코더의 hidden state들을 연결
decoder_concat_input = Concatenate(axis = -1, name='concat_layer')([decoder_outputs, attn_out])

# 디코더의 출력층
decoder_softmax_layer = Dense(tar_vocab, activation='softmax')
decoder_softmax_outputs = decoder_softmax_layer(decoder_concat_input)

# 모델 정의
model = Model([encoder_inputs, decoder_inputs], decoder_softmax_outputs)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 63)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 63, 128)      2163200     input_1[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 63, 256), (N 394240      embedding[0][0]                  
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
______________________________________________________________________________________________

In [21]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

In [22]:
checkpoint_path = "model/check.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 모델의 가중치를 저장하는 콜백 만들기
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience = 2)

model.save_weights(checkpoint_path.format(epoch=0))

history = model.fit(x = [encoder_input_train, decoder_input_train], y = decoder_target_train, \
          validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size = 256, callbacks=[es, cp_callback], epochs = 15)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 240535 samples, validate on 42456 samples
Epoch 1/15
240384/240535 [============================>.] - ETA: 0s - loss: 0.4319
Epoch 00001: saving model to check.ckpt
240535/240535 [==============================] - 363s 2ms/sample - loss: 0.4318 - val_loss: 0.2690
Epoch 2/15
240384/240535 [============================>.] - ETA: 0s - loss: 0.2523
Epoch 00002: saving model to check.ckpt
240535/240535 [==============================] - 359s 1ms/sample - loss: 0.2523 - val_loss: 0.2366
Epoch 3/15
240384/240535 [============================>.] - ETA: 0s - loss: 0.2314
Epoch 00003: saving model to check.ckpt
240535/240535 [==============================] - 346s 1ms/sample - loss: 0.2314 - val_loss: 0.2308
Epoch 4/15
240384/240535 [============================>.] - ETA: 0s - loss: 0.2220
Epoch 00004: saving model to check.ckpt
240535/240535 [==============================] - 342s 1ms/sample -

In [23]:
# 요약 모델(카테고리 추출 모델) 저장
model.save('model/model_cat.h5')

In [24]:
src_index_to_word = src_tokenizer.index_word # 원문 단어 집합에서 정수 -> 단어를 얻음
tar_word_to_index = tar_tokenizer.word_index # 요약 단어 집합에서 단어 -> 정수를 얻음
tar_index_to_word = tar_tokenizer.index_word # 요약 단어 집합에서 정수 -> 단어를 얻음

In [25]:
# 인코더 설계
encoder_model = Model(inputs=encoder_inputs, outputs=[encoder_outputs, state_h, state_c])

In [26]:
# 인코더 모델 저장
encoder_model.save('model/model_en.h5')

In [27]:
# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(hidden_size,))
decoder_state_input_c = Input(shape=(hidden_size,))

dec_emb2 = dec_emb_layer(decoder_inputs)
# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용. 이는 뒤의 함수 decode_sequence()에 구현
# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태인 state_h와 state_c를 버리지 않음.
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])

In [28]:
# 어텐션 함수
decoder_hidden_state_input = Input(shape=(text_max_len, hidden_size))
attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_input, decoder_outputs2])
decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_outputs2, attn_out_inf])

# 디코더의 출력층
decoder_outputs2 = decoder_softmax_layer(decoder_inf_concat) 

# 최종 디코더 모델
decoder_model = Model(
    [decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs2] + [state_h2, state_c2])

In [29]:
# 디코더 모델 저장
decoder_model.save('model/model_de.h5')

In [30]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    e_out, e_h, e_c = encoder_model.predict(input_seq)

     # <SOS>에 해당하는 토큰 생성
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = tar_word_to_index['sostoken']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition: # stop_condition이 True가 될 때까지 루프 반복
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = tar_index_to_word[sampled_token_index]

        if(sampled_token!='eostoken'):
            decoded_sentence += ' '+sampled_token

        #  <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_token == 'eostoken'  or len(decoded_sentence.split()) >= (summary_max_len-1)):
            stop_condition = True

        # 길이가 1인 타겟 시퀀스를 업데이트
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # 상태를 업데이트 합니다.
        e_h, e_c = h, c

    return decoded_sentence

In [31]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2text(input_seq):
    temp=''
    for i in input_seq:
        if(i!=0):
            temp = temp + src_index_to_word[i]+' '
    return temp

# 요약문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2summary(input_seq):
    temp=''
    for i in input_seq:
        if((i!=0 and i!=tar_word_to_index['sostoken']) and i!=tar_word_to_index['eostoken']):
            temp = temp + tar_index_to_word[i] + ' '
    return temp

In [32]:
# 예시 대화 데이터로 실험
pp_text=[]
fin_text=[]
k=0

f = open("data/testone.txt", 'r')
while True:
    line = f.readline()
    if not line: break
    pp_text.append(preprocess_sentence(line))
f.close()
tkn_text = src_tokenizer.texts_to_sequences(pp_text)

for i in range(len(tkn_text)):
        if not tkn_text[i-k]:
            del tkn_text[i-k]
            k+=1
        else:
            fin_text.append(np.array(tkn_text[i-k]))

In [33]:
fin_text = pad_sequences(fin_text, maxlen = text_max_len, padding='post')

In [34]:
# 문장 별로 주제 추출, 대화 속 가장 많이 나온 주제로 대화 주제 선정
cat_list = [" 개인 및 관계", " 미용과 건강", " 상거래 쇼핑", " 시사교육", " 식음료", " 여가 생활", " 일과 직업", " 주거와 생활", " 행사"]

def pick_cat(input_text):
    fir_list=[]
    for i in range(len(input_text)):
        fir_list.append(decode_sequence(input_text[i].reshape(1, text_max_len)))
    
    cat_count=[]
    for i in range(len(cat_list)):
        cat_count.append(fir_list.count(cat_list[i]))
        
    max_count = max(cat_count)
    max_name = cat_list[cat_count.index(max(cat_count))]

    cat_count[cat_count.index(max(cat_count))] = 0

    max2_count = max(cat_count)
    max2_name = cat_list[cat_count.index(max(cat_count))]

    cat_count[cat_count.index(max(cat_count))] = 0
    
    if max_name == " 개인 및 관계":
        if max_count >= max2_count*1.5:
            cat_result = max_name
        else:
            cat_result = max2_name
    else:
        cat_result = max_name
    
    return cat_result

In [35]:
pick_cat(fin_text)

' 개인 및 관계'